In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset

In [5]:
# Параметры
MODEL_NAME = "bert-base-uncased"
NUM_LABELS = 2  # Положительный или отрицательный отзыв
BATCH_SIZE = 8
EPOCHS = 3

In [6]:
# 1. Загрузка и подготовка данных
class ReviewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        item = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return {key: val.squeeze(0) for key, val in item.items()}

In [7]:
# Загрузка данных из CSV
df = pd.read_csv("/content/drive/MyDrive/bert/bert_poz_neg/train.csv", sep="\t")

In [8]:
df

,idx,Score,Text
0,0,Positive,В Альфа-Банке работает замечательная девушка -...
1,1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,2,Positive,Очень порадовала оперативность работы в банке....
3,3,Negative,Имела неосторожность оформить потреб. кредит в...
4,4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...,...
13994,13994,Positive,"О высокой надёжности МКБ, порядочности и добро..."
13995,13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...
13997,13997,Positive,Добрый день! Давно являюсь клиентом банка Русс...


In [9]:
df['Score'] = df['Score'].map({'Positive': 1, 'Negative': 0})

In [10]:
df

,idx,Score,Text
0,0,1,В Альфа-Банке работает замечательная девушка -...
1,1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,2,1,Очень порадовала оперативность работы в банке....
3,3,0,Имела неосторожность оформить потреб. кредит в...
4,4,0,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...,...
13994,13994,1,"О высокой надёжности МКБ, порядочности и добро..."
13995,13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,13996,1,Попала сегодня в очень неприятную ситуацию. Ре...
13997,13997,1,Добрый день! Давно являюсь клиентом банка Русс...


In [11]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Text'].tolist(),
    df['Score'].tolist(),
    test_size=0.2,
    random_state=42
)

In [12]:
# 2. Инициализация токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [14]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:
# Создание датасетов
train_dataset = ReviewsDataset(train_texts, train_labels, tokenizer)
val_dataset = ReviewsDataset(val_texts, val_labels, tokenizer)

In [16]:
train_dataset

In [17]:
# Коллектор данных для заполнения до max_length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [19]:
# 3. Функция вычисления метрик
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [27]:
# 4. Настройка Trainer
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    logging_dir=None,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
'''training_args = TrainingArguments(
  output_dir="cache",
  learning_rate=3e-4,
  per_device_train_batch_size=64,
  per_device_eval_batch_size=8,
  num_train_epochs=10,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
  save_total_limit=3,
  fp16=True,
  report_to = "none", # Use this for WandB etc
)'''

'training_args = TrainingArguments(\n  output_dir="cache",\n  learning_rate=3e-4,\n  per_device_train_batch_size=64,\n  per_device_eval_batch_size=8,\n  num_train_epochs=10,\n  weight_decay=0.01,\n  evaluation_strategy="epoch",\n  save_strategy="epoch",\n  load_best_model_at_end=True,\n  save_total_limit=3,\n  fp16=True,\n  report_to = "none", # Use this for WandB etc\n)'

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [29]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [30]:
# 5. Обучение модели
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.564500,0.548465,0.727857,0.713902,0.759828,0.736150
2,0.500100,0.578741,0.729286,0.681176,0.861330,0.760732
3,0.431600,0.528301,0.762500,0.774290,0.740529,0.757033


TrainOutput(global_step=4200, training_loss=0.4964015125093006, metrics={'train_runtime': 1090.3104, 'train_samples_per_second': 30.814, 'train_steps_per_second': 3.852, 'total_flos': 2209935531732480.0, 'train_loss': 0.4964015125093006, 'epoch': 3.0})

In [ ]:
'''# 6. Предсказания
texts_to_predict = ["Ваш отзыв здесь"]
predict_dataset = ReviewsDataset(texts_to_predict, [0] * len(texts_to_predict), tokenizer)
predictions = trainer.predict(predict_dataset)

# Вывод предсказаний
preds = predictions.predictions.argmax(-1)
print(["Positive" if label == 1 else "Negative" for label in preds])'''

In [33]:
test_df = pd.read_csv("/content/drive/MyDrive/bert/bert_poz_neg/test.csv", sep='\t')  # укажите путь к тестовому файлу

In [34]:
test_df

,idx,Text
0,13999,Замечательная история об открытии счета в банк...
1,14000,"23.07.14г. я досрочно внес сумму, указанную оп..."
2,14001,Была задолженность по кредитной карте. Банк по...
3,14002,Срочно понадобилась определенная денежная сумм...
4,14003,Здравствуйте. В местном отделении Бинбанка г. ...
...,...,...
5995,19994,Уважаемые представители Русфинанс Банка! Прошу...
5996,19995,04 июня сняли с дебетовой карты банка 10 000 T...
5997,19996,В апреле решил я купить шуруповерт. Посмотрел ...
5998,19997,Недавно взял потребкредит в этом банке. Все че...


In [35]:
test_texts = test_df['Text'].tolist()

In [37]:
test_texts

['Замечательная история об открытии счета в банке Авангард в г. Ростов-на-Дону филиал Центральный, что на пр. Шолохова.Прелюдия.Мой друг из Москвы поделился, что в банке очень хорошие условия обслуживания, в частности снятие наличных и внесение на расчётный счет делается через банкомат.Кроме этого при внесении, если через банк-клиент уведомляешь банк за день, то и комиссия не взимается. Залез на сайт - почитал. Отметил ещё одно достоинство - интернет-Эквайринг.Отличный современный банк сделал я поспешный вывод. И решил открыть счета моих новых организаций в нем.Банк Авангард.Открытие счета - 1004 рубля, да. не 1000, которые написано на сайте, но и комиссия за внесение наличных. Для открытия счета нужна заверенная нотариально банковская карточка. Ладно, 200 рублей за каждую подпись. Заполняем пакет документов - приносим в банк, принимает милая девушка Татьяна. Приходите, завтра заберете документы на открытый счет. Странно, но ладно обычно счет открывают сразу в моем присутствии. Ну пред

In [39]:
# Подготовка тестового датасета
test_dataset = ReviewsDataset(test_texts, [0] * len(test_texts), tokenizer)

In [40]:
test_dataset

In [41]:
# Предсказания на тестовом наборе
predictions = trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [42]:
predictions

PredictionOutput(predictions=array([[ 0.5760173 , -0.3530233 ],
       [ 1.613053  , -1.392591  ],
       [-0.09921568,  0.23480934],
       ...,
       [ 1.4733412 , -1.2277236 ],
       [ 0.8630549 , -0.61837876],
       [ 0.7802191 , -0.46787784]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 1.2113804817199707, 'test_accuracy': 0.5218333333333334, 'test_precision': 0.0, 'test_recall': 0.0, 'test_f1': 0.0, 'test_runtime': 83.3941, 'test_samples_per_second': 71.948, 'test_steps_per_second': 8.993})

In [43]:
preds = predictions.predictions.argmax(-1)

In [44]:
preds

array([0, 0, 1, ..., 0, 0, 0])

In [45]:
# Преобразование предсказаний в метки
test_df['Score'] = ["Positive" if label == 1 else "Negative" for label in preds]

In [46]:
test_df

,idx,Text,Score
0,13999,Замечательная история об открытии счета в банк...,Negative
1,14000,"23.07.14г. я досрочно внес сумму, указанную оп...",Negative
2,14001,Была задолженность по кредитной карте. Банк по...,Positive
3,14002,Срочно понадобилась определенная денежная сумм...,Negative
4,14003,Здравствуйте. В местном отделении Бинбанка г. ...,Negative
...,...,...,...
5995,19994,Уважаемые представители Русфинанс Банка! Прошу...,Negative
5996,19995,04 июня сняли с дебетовой карты банка 10 000 T...,Negative
5997,19996,В апреле решил я купить шуруповерт. Посмотрел ...,Negative
5998,19997,Недавно взял потребкредит в этом банке. Все че...,Negative


In [47]:
# Сохранение результатов в submission.csv
submission_df = test_df[['Text', 'Score']]  # Оставляем только нужные столбцы

In [48]:
submission_df

,Text,Score
0,Замечательная история об открытии счета в банк...,Negative
1,"23.07.14г. я досрочно внес сумму, указанную оп...",Negative
2,Была задолженность по кредитной карте. Банк по...,Positive
3,Срочно понадобилась определенная денежная сумм...,Negative
4,Здравствуйте. В местном отделении Бинбанка г. ...,Negative
...,...,...
5995,Уважаемые представители Русфинанс Банка! Прошу...,Negative
5996,04 июня сняли с дебетовой карты банка 10 000 T...,Negative
5997,В апреле решил я купить шуруповерт. Посмотрел ...,Negative
5998,Недавно взял потребкредит в этом банке. Все че...,Negative


In [50]:
sample = pd.read_csv('/content/drive/MyDrive/bert/bert_poz_neg/sample_submission.csv', sep='\t')
sample

,idx,Score
0,13999,Positive
1,14000,Positive
2,14001,Positive
3,14002,Positive
4,14003,Positive
...,...,...
5995,19994,Positive
5996,19995,Positive
5997,19996,Positive
5998,19997,Positive


In [51]:
submission1 = pd.DataFrame({'idx': sample['idx'], 'Score': test_df['Score']})

In [52]:
submission1

,idx,Score
0,13999,Negative
1,14000,Negative
2,14001,Positive
3,14002,Negative
4,14003,Negative
...,...,...
5995,19994,Negative
5996,19995,Negative
5997,19996,Negative
5998,19997,Negative


In [49]:
submission_df.to_csv("bert.csv", index=False, sep="\t")

In [53]:
submission1.to_csv("bert1.csv", index=False, sep="\t")